In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT 
    wp.cuscode AS cuscode,
    TRIM(wp.name) AS name,
    TRIM(wp.fin_sale) AS fin_sale,
    TRIM(wp.fin_new_group) AS fin_new_group,
    TRIM(wp.type_agent) AS type_agent,

    ic_ins.card_no AS card_no_INS,
    ic_ins.type AS type_INS,
    ic_ins.revoke_type_code AS revoke_type_INS,

    ic_life.card_no AS card_no_LIFE,
    ic_life.type AS type_LIFE,
    ic_life.revoke_type_code AS revoke_type_LIFE,

    CASE 
        WHEN DATEDIFF(CURRENT_DATE(), wp.date_active) <= 30 THEN 'active'
        ELSE 'nonactive'
    END AS status,

    CASE 
        WHEN wp.file_id_card IS NOT NULL AND wp.file_id_card <> '' THEN 'yes'
        ELSE 'no'
    END AS `บัตรประชาชน`,

    CASE 
        WHEN wp.file_bookbank IS NOT NULL AND wp.file_bookbank <> '' THEN 'yes'
        ELSE 'no'
    END AS `หน้าบัญชีธนาคาร`,

    CASE 
        WHEN wp.file_id_home IS NOT NULL AND wp.file_id_home <> '' THEN 'yes'
        ELSE 'no'
    END AS `ทะเบียนบ้าน`

FROM wp_users wp
LEFT JOIN tbl_ins_card ic_ins
    ON ic_ins.cuscode = wp.cuscode AND ic_ins.ins_type = 'INS'
LEFT JOIN tbl_ins_card ic_life
    ON ic_life.cuscode = wp.cuscode AND ic_life.ins_type = 'LIFE'
WHERE wp.cuscode LIKE 'FNG%'
    AND wp.name NOT LIKE '%ทดสอบ%'
    AND wp.name NOT LIKE '%test%'
    AND wp.name NOT LIKE '%เทสระบบ%'
    AND wp.name NOT LIKE '%Tes ระบบ%'
    AND wp.name NOT LIKE '%ทด่ท%'
    AND wp.name NOT LIKE '%ทด สอบ%'
    AND wp.fin_new_group NOT LIKE '%test%'
    AND wp.fin_sale NOT LIKE '%IT-TESTER%';
"""

df = pd.read_sql(query, engine)
df


,cuscode,name,fin_sale,fin_new_group,type_agent,card_no_INS,type_INS,revoke_type_INS,card_no_LIFE,type_LIFE,revoke_type_LIFE,status,บัตรประชาชน,หน้าบัญชีธนาคาร,ทะเบียนบ้าน
0,FNG1911-0037,ปัญญา,,FIN-East1,None,None,None,None,None,None,None,nonactive,yes,no,no
1,FNG1912-0003,ณัฏฐา รังษีกุลพิพัฒน์,Sale-Mem10,FIN-Northeast1,None,None,None,None,None,None,None,nonactive,yes,no,no
2,FNG1912-0004,ปัญญากร,Sale-Mem10,FIN-BKK1,None,6204022777,B,,6503003176,B,,nonactive,yes,no,no
3,FNG1912-0005,เจษฎาพงษ์ ชาติศรี,Sale-Mem10,FIN-East1,sell,None,None,None,None,None,None,nonactive,yes,yes,no
4,FNG1912-0006,เก่งชัย แปะแนะ,Sale-Mem10,FIN-South1,None,None,None,None,None,None,None,nonactive,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139422,FNG25-140021,อรสา ศรีเสมอ,Sale-Mem50,FIN-Online1,buy,None,None,None,None,None,None,active,no,no,no
139423,FNG25-140022,ณชนก เอื้อจำนงค์,Sale-Mem41,FIN-BKK1,sell,6801036921,A,None,None,None,None,active,yes,yes,yes
139424,FNG25-140023,นฤนาท มากสอน,Sale-Mem10,FIN-Online1,buy,None,None,None,None,None,None,nonactive,no,no,no
139425,FNG25-140024,ศราวุฒิ บุญนาค,FIN-Online,FIN-Online1,buy,None,None,None,None,None,None,active,no,no,no
